In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import sweetviz
from sklearn.model_selection import train_test_split
from feature_engine.outliers import Winsorizer
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [2]:
df1 = pd.read_csv('Summary of Weather.csv')
df2 = pd.read_csv('Weather Station Locations.csv')
df = pd.concat([df1, df2], axis=1)
df.head()

C:\Users\Dell\AppData\Local\Temp\ipykernel_13488\92920450.py:1: DtypeWarning: Columns (7,8,18,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('Summary of Weather.csv')


,STA,Date,Precip,WindGustSpd,MaxTemp,MinTemp,MeanTemp,Snowfall,PoorWeather,YR,...,RVG,WTE,WBAN,NAME,STATE/COUNTRY ID,LAT,LON,ELEV,Latitude,Longitude
0,10001,1942-7-1,1.016,NaN,25.555556,22.222222,23.888889,0.0,NaN,42,...,NaN,NaN,33013.0,AIN EL,AL,3623N,00637E,611.0,36.383333,6.650000
1,10001,1942-7-2,0,NaN,28.888889,21.666667,25.555556,0.0,NaN,42,...,NaN,NaN,33031.0,LA SENIA,AL,3537N,00037E,88.0,35.616667,0.583333
2,10001,1942-7-3,2.54,NaN,26.111111,22.222222,24.444444,0.0,NaN,42,...,NaN,NaN,33023.0,MAISON BLANCHE,AL,3643N,00314E,23.0,36.716667,3.216667
3,10001,1942-7-4,2.54,NaN,26.666667,22.222222,24.444444,0.0,NaN,42,...,NaN,NaN,33044.0,TELERGMA,AL,3607N,00621E,754.0,36.116667,6.416667
4,10001,1942-7-5,0,NaN,26.666667,21.666667,24.444444,0.0,NaN,42,...,NaN,NaN,12001.0,TINDOUF,AL,2741N,00809W,443.0,27.683333,-8.083333


In [3]:
#EDA
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119040 entries, 0 to 119039
Data columns (total 39 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   STA               119040 non-null  int64  
 1   Date              119040 non-null  object 
 2   Precip            119040 non-null  object 
 3   WindGustSpd       532 non-null     float64
 4   MaxTemp           119040 non-null  float64
 5   MinTemp           119040 non-null  float64
 6   MeanTemp          119040 non-null  float64
 7   Snowfall          117877 non-null  object 
 8   PoorWeather       34237 non-null   object 
 9   YR                119040 non-null  int64  
 10  MO                119040 non-null  int64  
 11  DA                119040 non-null  int64  
 12  PRCP              117108 non-null  object 
 13  DR                533 non-null     float64
 14  SPD               532 non-null     float64
 15  MAX               118566 non-null  float64
 16  MIN               11

In [4]:
df.describe()

,STA,WindGustSpd,MaxTemp,MinTemp,MeanTemp,YR,MO,DA,DR,SPD,...,PGT,SD3,RHX,RHN,RVG,WTE,WBAN,ELEV,Latitude,Longitude
count,119040.000000,532.000000,119040.000000,119040.000000,119040.000000,119040.000000,119040.000000,119040.000000,533.000000,532.000000,...,525.000000,0.0,0.0,0.0,0.0,0.0,161.000000,161.000000,161.000000,161.000000
mean,29659.435795,37.774534,27.045111,17.789511,22.411631,43.805284,6.726016,15.797530,26.998124,20.396617,...,12.085333,NaN,NaN,NaN,NaN,NaN,33421.440994,439.850932,20.347205,8.364182
std,20953.209402,10.297808,8.717817,8.334572,8.297982,1.136718,3.425561,8.794541,15.221732,5.560371,...,5.731328,NaN,NaN,NaN,NaN,NaN,20826.534394,1745.422339,21.042992,87.420419
min,10001.000000,18.520000,-33.333333,-38.333333,-35.555556,40.000000,1.000000,1.000000,2.000000,10.000000,...,0.000000,NaN,NaN,NaN,NaN,NaN,10001.000000,1.000000,-27.600000,-175.000000
25%,11801.000000,29.632000,25.555556,15.000000,20.555556,43.000000,4.000000,8.000000,11.000000,16.000000,...,8.500000,NaN,NaN,NaN,NaN,NaN,13601.000000,9.000000,8.233333,-60.850000
50%,22508.000000,37.040000,29.444444,21.111111,25.555556,44.000000,7.000000,16.000000,32.000000,20.000000,...,11.600000,NaN,NaN,NaN,NaN,NaN,32906.000000,27.000000,22.333333,9.083333
75%,33501.000000,43.059000,31.666667,23.333333,27.222222,45.000000,10.000000,23.000000,34.000000,23.250000,...,15.000000,NaN,NaN,NaN,NaN,NaN,34150.000000,98.000000,36.116667,67.250000
max,82506.000000,75.932000,50.000000,34.444444,40.000000,45.000000,12.000000,31.000000,78.000000,41.000000,...,23.900000,NaN,NaN,NaN,NaN,NaN,82506.000000,9999.000000,67.016667,177.366667


In [5]:
# Convert columns to float
df['Precip'] = pd.to_numeric(df['Precip'], errors='coerce')
df['Snowfall'] = pd.to_numeric(df['Snowfall'], errors='coerce')
df['PRCP'] = pd.to_numeric(df['PRCP'], errors='coerce')



In [6]:
#feature engineering

columns_to_drop = [
    'STA',                # Station ID (not needed for analysis)
    'Date',               # Date (split into year/month/day if necessary)
    'FT',                 # Unknown
    'FB',                 # Unknown
    'FTI',                # Unknown
    'ITH',                # Unknown
    'PGT',                # Unknown
    'TSHDSBRSGF',         # Thunderstorm indicator (not needed for mean temperature analysis)
    'SD3',                # Unknown
    'RHX',                # Unknown
    'RHN',                # Unknown
    'RVG',                # Unknown
    'WTE',                # Unknown
    'PoorWeather',       # High missing values
    'SNF',               # High missing values
    'NAME',              # High missing values
    'STATE/COUNTRY ID',  # High missing values
    'LAT',               # High missing values
    'LON',               # High missing values
    'WindGustSpd',       # Low variance
    'DR',                # Low variance
    'SPD',               # Low variance
    'SND',               # Low variance
    'WBAN'               # Low variance
]

# Drop the specified columns
df.drop(columns=columns_to_drop, inplace=True)

# Define X (features) and y (target variable)
X = df.drop(columns='MeanTemp')  # All features except MeanTemp
y = df['MeanTemp']                # Target variable for analysis


In [7]:
X.isna().sum()

Precip        16753
MaxTemp           0
MinTemp           0
Snowfall       1207
YR                0
MO                0
DA                0
PRCP          18685
MAX             474
MIN             468
MEA             498
ELEV         118879
Latitude     118879
Longitude    118879
dtype: int64

In [8]:
X.duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
119035    False
119036    False
119037    False
119038    False
119039    False
Length: 119040, dtype: bool

In [9]:
numeric_features = X.select_dtypes(exclude = ['object']).columns
categorical_columns = X.select_dtypes(include = ['object']).columns


In [10]:
#data preprocessing
#outliers
# Specify the columns as a list
variables = ['MaxTemp', 'MinTemp', 'YR', 'MO',
       'DA', 'MAX', 'MIN', 'MEA']

# Pass the list to the Winsorizer
winsor = Winsorizer(capping_method='iqr',
                    tail='both',
                    fold=1.5,
                    variables=variables)


In [11]:
#imputation
num_pipeline = Pipeline(steps = [('Impute', SimpleImputer(strategy = 'mean'))])
outlier_pipeline = Pipeline(steps = [('winsor', winsor)])

In [12]:
#OneHotEncoding
from sklearn.preprocessing import OneHotEncoder
encode_pipeline = Pipeline(steps = [('OneHot', OneHotEncoder(sparse_output=False, drop = 'first'))])

In [13]:
num_preprocessor = ColumnTransformer(transformers = [('num', num_pipeline, numeric_features)])

In [14]:
win_preprocessor = ColumnTransformer(transformers = [('wins', outlier_pipeline, numeric_features)])

In [15]:
encode_preprocessor = ColumnTransformer(transformers = [('encode', encode_pipeline, categorical_columns)])

In [16]:
impute_data = num_preprocessor.fit_transform(X)
impute_df = pd.DataFrame(impute_data, columns = numeric_features)
df1

,STA,Date,Precip,WindGustSpd,MaxTemp,MinTemp,MeanTemp,Snowfall,PoorWeather,YR,...,FB,FTI,ITH,PGT,TSHDSBRSGF,SD3,RHX,RHN,RVG,WTE
0,10001,1942-7-1,1.016,NaN,25.555556,22.222222,23.888889,0.0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10001,1942-7-2,0,NaN,28.888889,21.666667,25.555556,0.0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10001,1942-7-3,2.54,NaN,26.111111,22.222222,24.444444,0.0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10001,1942-7-4,2.54,NaN,26.666667,22.222222,24.444444,0.0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10001,1942-7-5,0,NaN,26.666667,21.666667,24.444444,0.0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119035,82506,1945-12-27,0,NaN,28.333333,18.333333,23.333333,0.0,NaN,45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119036,82506,1945-12-28,9.906,NaN,29.444444,18.333333,23.888889,0.0,1.0,45,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
119037,82506,1945-12-29,0,NaN,28.333333,18.333333,23.333333,0.0,1.0,45,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
119038,82506,1945-12-30,0,NaN,28.333333,18.333333,23.333333,0.0,NaN,45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
wins_data = win_preprocessor.fit_transform(impute_df)
wins_df = pd.DataFrame(wins_data, columns = impute_df.columns)
wins_df

,Precip,MaxTemp,MinTemp,Snowfall,YR,MO,DA,PRCP,MAX,MIN,MEA,ELEV,Latitude,Longitude
0,1.016,25.555556,22.222222,0.0,42.0,7.0,1.0,0.04,78.0,72.0,75.0,611.000000,36.383333,6.650000
1,0.000,28.888889,21.666667,0.0,42.0,7.0,2.0,0.00,84.0,71.0,78.0,88.000000,35.616667,0.583333
2,2.540,26.111111,22.222222,0.0,42.0,7.0,3.0,0.10,79.0,72.0,76.0,23.000000,36.716667,3.216667
3,2.540,26.666667,22.222222,0.0,42.0,7.0,4.0,0.10,80.0,72.0,76.0,754.000000,36.116667,6.416667
4,0.000,26.666667,21.666667,0.0,42.0,7.0,5.0,0.00,80.0,71.0,76.0,443.000000,27.683333,-8.083333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119035,0.000,28.333333,18.333333,0.0,45.0,12.0,27.0,0.00,83.0,65.0,74.0,439.850932,20.347205,8.364182
119036,9.906,29.444444,18.333333,0.0,45.0,12.0,28.0,0.39,85.0,65.0,75.0,439.850932,20.347205,8.364182
119037,0.000,28.333333,18.333333,0.0,45.0,12.0,29.0,0.00,83.0,65.0,74.0,439.850932,20.347205,8.364182
119038,0.000,28.333333,18.333333,0.0,45.0,12.0,30.0,0.00,83.0,65.0,74.0,439.850932,20.347205,8.364182


In [18]:
encode_data = encode_preprocessor.fit_transform(X[categorical_columns])

# Retrieve feature names and create the DataFrame
columns = encode_preprocessor.get_feature_names_out(categorical_columns)
df3 = pd.DataFrame(encode_data, columns=columns)
df3

""
0
1
2
3
4
...
119035
119036
119037
119038


In [19]:
from sklearn.preprocessing import MinMaxScaler
scale_pipeline = Pipeline(steps = [('minmax', MinMaxScaler())])
scale_preprocessor = ColumnTransformer(transformers = [('minmax', scale_pipeline, numeric_features)])

In [20]:
scale_data = scale_preprocessor.fit_transform(wins_df)
df4 = pd.DataFrame(scale_data, columns = wins_df.columns)
df4

,Precip,MaxTemp,MinTemp,Snowfall,YR,MO,DA,PRCP,MAX,MIN,MEA,ELEV,Latitude,Longitude
0,0.003306,0.375000,0.617391,0.0,0.4,0.545455,0.000000,0.003306,0.375000,0.617391,0.500000,0.061012,0.676237,0.515514
1,0.000000,0.511364,0.600000,0.0,0.4,0.545455,0.033333,0.000000,0.511364,0.600000,0.562500,0.008702,0.668135,0.498297
2,0.008264,0.397727,0.617391,0.0,0.4,0.545455,0.066667,0.008264,0.397727,0.617391,0.520833,0.002200,0.679760,0.505771
3,0.008264,0.420455,0.617391,0.0,0.4,0.545455,0.100000,0.008264,0.420455,0.617391,0.520833,0.075315,0.673419,0.514852
4,0.000000,0.420455,0.600000,0.0,0.4,0.545455,0.133333,0.000000,0.420455,0.600000,0.520833,0.044209,0.584287,0.473702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119035,0.000000,0.488636,0.495652,0.0,1.0,1.000000,0.866667,0.000000,0.488636,0.495652,0.479167,0.043894,0.506752,0.520379
119036,0.032231,0.534091,0.495652,0.0,1.0,1.000000,0.900000,0.032231,0.534091,0.495652,0.500000,0.043894,0.506752,0.520379
119037,0.000000,0.488636,0.495652,0.0,1.0,1.000000,0.933333,0.000000,0.488636,0.495652,0.479167,0.043894,0.506752,0.520379
119038,0.000000,0.488636,0.495652,0.0,1.0,1.000000,0.966667,0.000000,0.488636,0.495652,0.479167,0.043894,0.506752,0.520379


In [21]:
clean_df = pd.concat([df4, df3], axis=1)  # Use axis=1 to concatenate columns side by side
clean_df


,Precip,MaxTemp,MinTemp,Snowfall,YR,MO,DA,PRCP,MAX,MIN,MEA,ELEV,Latitude,Longitude
0,0.003306,0.375000,0.617391,0.0,0.4,0.545455,0.000000,0.003306,0.375000,0.617391,0.500000,0.061012,0.676237,0.515514
1,0.000000,0.511364,0.600000,0.0,0.4,0.545455,0.033333,0.000000,0.511364,0.600000,0.562500,0.008702,0.668135,0.498297
2,0.008264,0.397727,0.617391,0.0,0.4,0.545455,0.066667,0.008264,0.397727,0.617391,0.520833,0.002200,0.679760,0.505771
3,0.008264,0.420455,0.617391,0.0,0.4,0.545455,0.100000,0.008264,0.420455,0.617391,0.520833,0.075315,0.673419,0.514852
4,0.000000,0.420455,0.600000,0.0,0.4,0.545455,0.133333,0.000000,0.420455,0.600000,0.520833,0.044209,0.584287,0.473702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119035,0.000000,0.488636,0.495652,0.0,1.0,1.000000,0.866667,0.000000,0.488636,0.495652,0.479167,0.043894,0.506752,0.520379
119036,0.032231,0.534091,0.495652,0.0,1.0,1.000000,0.900000,0.032231,0.534091,0.495652,0.500000,0.043894,0.506752,0.520379
119037,0.000000,0.488636,0.495652,0.0,1.0,1.000000,0.933333,0.000000,0.488636,0.495652,0.479167,0.043894,0.506752,0.520379
119038,0.000000,0.488636,0.495652,0.0,1.0,1.000000,0.966667,0.000000,0.488636,0.495652,0.479167,0.043894,0.506752,0.520379


In [22]:
#data analysis  after pre-processing
X[numeric_features].skew()

Precip        6.695241
MaxTemp      -2.080644
MinTemp      -1.889984
Snowfall     16.454033
YR           -0.965982
MO           -0.074214
DA           -0.002949
PRCP          6.633991
MAX          -1.881670
MIN          -1.820894
MEA          -2.026856
ELEV          5.200551
Latitude     -0.296729
Longitude     0.024284
dtype: float64

In [23]:
#kurtosis
X[numeric_features].kurt()

Precip        72.363639
MaxTemp        6.330238
MinTemp        4.517975
Snowfall     333.090877
YR             0.763176
MO            -1.185754
DA            -1.191827
PRCP          71.076188
MAX            5.314212
MIN            4.310761
MEA            5.442179
ELEV          26.146312
Latitude      -0.224391
Longitude     -0.566349
dtype: float64

In [24]:
#corelation
X[numeric_features].corr()

,Precip,MaxTemp,MinTemp,Snowfall,YR,MO,DA,PRCP,MAX,MIN,MEA,ELEV,Latitude,Longitude
Precip,1.000000,-0.003083,0.098579,-0.010480,0.004260,0.041121,0.000810,1.000000,-0.004075,0.102444,0.050442,0.364826,-0.122885,-0.098730
MaxTemp,-0.003083,1.000000,0.878384,-0.336999,0.039585,0.031346,-0.005130,-0.006801,1.000000,0.870594,0.968247,0.015578,0.021637,0.003554
MinTemp,0.098579,0.878384,1.000000,-0.320022,-0.020733,0.069078,-0.002576,0.097451,0.872096,1.000000,0.966441,-0.027406,-0.050511,0.252694
Snowfall,-0.010480,-0.336999,-0.320022,1.000000,-0.000890,-0.045052,-0.000271,-0.010960,-0.357927,-0.333037,-0.357768,NaN,NaN,NaN
YR,0.004260,0.039585,-0.020733,-0.000890,1.000000,-0.144360,-0.011196,0.001050,0.013986,-0.043323,-0.014723,NaN,NaN,NaN
MO,0.041121,0.031346,0.069078,-0.045052,-0.144360,1.000000,0.006563,0.040774,0.034329,0.072453,0.054847,0.079264,0.097200,0.001902
DA,0.000810,-0.005130,-0.002576,-0.000271,-0.011196,0.006563,1.000000,0.001086,-0.003299,-0.000739,-0.002094,-0.077952,-0.107884,-0.168575
PRCP,1.000000,-0.006801,0.097451,-0.010960,0.001050,0.040774,0.001086,1.000000,-0.008176,0.101307,0.047835,0.364826,-0.122885,-0.098730
MAX,-0.004075,1.000000,0.872096,-0.357927,0.013986,0.034329,-0.003299,-0.008176,1.000000,0.873719,0.968247,0.015578,0.021637,0.003554
MIN,0.102444,0.870594,1.000000,-0.333037,-0.043323,0.072453,-0.000739,0.101307,0.873719,1.000000,0.966609,-0.027406,-0.050511,0.252694


In [25]:
auto_df = sweetviz.analyze(df)
auto_df.show_html('report.html')

                                             |                                             | [  0%]   00:00 ->…

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
---
(likely due to only having a single row, containing non-NaN values for both correlated features)
Affected correlations:['Snowfall/ELEV', 'Snowfall/Latitude', 'Snowfall/Longitude', 'ELEV/Snowfall', 'Latitude/Snowfall', 'Longitude/Snowfall']


In [26]:
#After data preprocessing
auto_df = sweetviz.analyze(clean_df)
auto_df.show_html('report.html')

                                             |                                             | [  0%]   00:00 ->…

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [27]:
# Partition the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(clean_df, y, test_size=0.2, random_state=42)

In [28]:
# Combine training features and target variable into a single DataFrame for statsmodels
train_data = pd.concat([x_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)

In [29]:
#model building
# # Linear Regression using statsmodels package
# Fit the Linear Regression model using statsmodels
model = smf.ols('MeanTemp ~ ' + ' + '.join(train_data.columns[:-1]), data=train_data).fit()  # Explicitly specifying features
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               MeanTemp   R-squared:                       0.929
Model:                            OLS   Adj. R-squared:                  0.929
Method:                 Least Squares   F-statistic:                 8.937e+04
Date:                Mon, 28 Oct 2024   Prob (F-statistic):               0.00
Time:                        17:44:37   Log-Likelihood:            -2.1066e+05
No. Observations:               95232   AIC:                         4.214e+05
Df Residuals:                   95217   BIC:                         4.215e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.6806      0.657      7.125      0.000       3.393       5.968
Precip         5.4580      4.595      1.188      0.235      -3.548      14.464
MaxTemp       56.4746      0.736     76.719      0.000      55.032      57.917
MinTemp       32.1360      0.681     47.156      0.000      30.800      33.472
Snowfall     -34.0888      0.246   -138.446      0.000     -34.571     -33.606
YR             0.5799      0.032     18.005      0.000       0.517       0.643
MO            -0.0394      0.023     -1.688      0.091      -0.085       0.006
DA            -0.0264      0.024     -1.079      0.281      -0.074       0.022
PRCP          -5.6901      4.601     -1.237      0.216     -14.707       3.327
MAX          -24.0133      0.770    -31.178      0.000     -25.523     -22.504
MIN           11.0656      0.738     15.000      0.000       9.620      12.511
MEA          -39.9020      0.479    -83.287      0.000     -40.841     -38.963
ELEV          -0.4345      1.125     -0.386      0.699      -2.640       1.771
Latitude      -0.4664      0.848     -0.550      0.582      -2.129       1.196
Longitude     -0.1017      0.798     -0.127      0.899      -1.665       1.462
==============================================================================
Omnibus:                   101536.012   Durbin-Watson:                   1.997
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         15889231.830
Skew:                          -5.134   Prob(JB):                         0.00
Kurtosis:                      65.441   Cond. No.                     1.79e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.79e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [30]:
# Make predictions using the trained model
pred1 = model.predict(pd.DataFrame(train_data))  # Using the training data for predictions
pred1

0        15.871113
1        22.964542
2        26.906657
3        26.642444
4        26.384691
           ...    
95227    23.368232
95228    27.643786
95229    28.278393
95230    28.962991
95231    26.778226
Length: 95232, dtype: float64

In [31]:
# Error calculation (error = AV - PV)
# Error calculation for training data
res1 = train_data['MeanTemp'] - pred1  # Residuals for training data
res_sqr1 = res1 ** 2                    # Squared residuals for training data

# Calculate error metrics for training data
train_mean_residual = np.mean(res1)      # Mean of residuals for training data
train_mse = np.mean(res_sqr1)            # Mean Squared Error for training data
train_rmse = np.sqrt(train_mse)           # Root Mean Squared Error for training data

In [32]:
# Combine testing features and target variable into a single DataFrame for statsmodels
test_data = pd.concat([x_test.reset_index(drop=True), y_test.reset_index(drop=True)], axis=1)

In [33]:
# Predict on test data
test_pred = model.predict(pd.DataFrame(test_data))
test_pred

0        23.502217
1        24.837685
2        25.032872
3        30.762249
4        34.032897
           ...    
23803     4.824505
23804     4.788752
23805    23.530377
23806    24.857008
23807    26.216868
Length: 23808, dtype: float64

In [34]:
# Error calculation for testing data
test_res = test_data['MeanTemp'] - test_pred  # Residuals for testing data
test_res_sqr = test_res ** 2                    # Squared residuals for testing data

# Calculate error metrics for testing data
test_mean_residual = np.mean(test_res)        # Mean of residuals for testing data
test_mse = np.mean(test_res_sqr)              # Mean Squared Error for testing data
test_rmse = np.sqrt(test_mse)                  # Root Mean Squared Error for testing data

In [35]:
# Create a DataFrame to compare training and testing error metrics
error_metrics = pd.DataFrame({
    'Dataset': ['Training', 'Testing'],
    'Mean of Residuals': [train_mean_residual, test_mean_residual],
    'Mean Squared Error (MSE)': [train_mse, test_mse],
    'Root Mean Squared Error (RMSE)': [train_rmse, test_rmse]
})
error_metrics

,Dataset,Mean of Residuals,Mean Squared Error (MSE),Root Mean Squared Error (RMSE)
0,Training,2.489959e-14,4.885599,2.210339
1,Testing,-2.725618e-03,5.098710,2.258032


In [36]:
#model2
from sklearn.tree import DecisionTreeRegressor
model2 = DecisionTreeRegressor()
model2.fit(x_train, y_train)
dt_pred = model2.predict(pd.DataFrame(x_train))
dt_pred

array([17.22222222, 23.33333333, 26.11111111, ..., 27.77777778,
       27.77777778, 26.66666667])

In [37]:
# Error calculation (error = AV - PV)
# Error calculation for training data
dt_res1 = train_data['MeanTemp'] - dt_pred  # Residuals for training data
dt_res_sqr1 = dt_res1 ** 2                    # Squared residuals for training data
# Calculate error metrics for training data
dt_train_mean_residual = np.mean(res1)      # Mean of residuals for training data
dt_train_mse = np.mean(res_sqr1)            # Mean Squared Error for training data
dt_train_rmse = np.sqrt(train_mse)           # Root Mean Squared Error for training data

In [38]:
#testing prediction
test_pred2 = model2.predict(pd.DataFrame(x_test))
test_pred2

array([24.44444444, 25.        , 24.44444444, ..., 24.44444444,
       25.        , 26.66666667])

In [39]:
# Error calculation for testing data
dt_test_res2 = test_data['MeanTemp'] - test_pred2  # Residuals for testing data
dt_test_res_sqr = dt_test_res2 ** 2                    # Squared residuals for testing data

# Calculate error metrics for testing data
dt_test_mean_residual = np.mean(test_res)        # Mean of residuals for testing data
dt_test_mse = np.mean(test_res_sqr)              # Mean Squared Error for testing data
dt_test_rmse = np.sqrt(test_mse)                  # Root Mean Squared Error for testing data

In [40]:
# Create a DataFrame to compare training and testing error metrics
error_metrics2 = pd.DataFrame({
    'Dataset': ['Training', 'Testing'],
    'Mean of Residuals': [dt_train_mean_residual, dt_test_mean_residual],
    'Mean Squared Error (MSE)': [dt_train_mse, dt_test_mse],
    'Root Mean Squared Error (RMSE)': [dt_train_rmse, dt_test_rmse]
})
error_metrics2

,Dataset,Mean of Residuals,Mean Squared Error (MSE),Root Mean Squared Error (RMSE)
0,Training,2.489959e-14,4.885599,2.210339
1,Testing,-2.725618e-03,5.098710,2.258032
